# Kerasでのネットワーク実装

手作りのニューラルネットワーク専用のフレームワーク「Keras」で構築しよう。まず、必要なパッケージを追加しよう

In [4]:
import numpy as np

# 入力、結合、活性化の「レイヤ」
from keras.layers import Input, Dense, Activation

# モデル
from keras.models import Model

# 活性化関数
from keras.activations import sigmoid

## モデル構築

Kerasのモデルは「レイヤ」で構築する。ネットワークの各パーツにたいして、適するレイヤがある：

- Input: 入力レイヤ。入力の数を定義する
- Dense: 全結合レイヤ。前レイヤの出力と本レイヤの出力をすべてつながるり、荷重を与える
- Activation：活性レイヤ。前レイヤの出力の出力に活性化関数で処理する

In [7]:
def create_model():

    # 入力レイヤ
    x = Input(shape=(2,), dtype=float)

    # 全結合レイヤ（荷重）出力が１つ
    w = Dense(units=1)(x)

    # 活性化レイヤ。活性化関数はsigmoid
    y = Activation(sigmoid)(w)

    # モデル
    model = Model(inputs=x, outputs=y)

    return model

In [8]:
# 確認しましょう
model = create_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

上記の出力を解析してみると…

`input_1 (InputLayer)        [(None, 2)]               0 `

入力レイヤは入力の数は２つがある。`None`はデータの数を示している。ANDとORの課題で、データの数は４点があるが、現時点でまだ確定ではないので`None`として表示されている。入力レイヤは学習できるパラメータはないので、パラメータの数は0である。

`dense (Dense)               (None, 1)                 3`

全結合レイヤの出力は１つしかない。上記と同様、まだ確定されていないデータの数は`None`で表示。なお、パラメータの数は3となる（荷重２つとバイアス１つ）

`activation (Activation)     (None, 1)                 0`

最後に活性化関数があり、出力は１つです。前の出力を処理するだけで、学習パラメータがない。

In [9]:
# 中身を確認
print (model.get_weights())

[array([[-0.6530792 ],
       [ 0.98512757]], dtype=float32), array([0.], dtype=float32)]


## 推論
このまま（学習なし）で推論をしてみると：

In [10]:
x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=float)

y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
[[0.5       ]
 [0.72812444]
 [0.342296  ]
 [0.5822577 ]]


## 学習

推論はまったく当たらないので、学習させましょう！

まず、kerasでは、そのまま学習できない。その先、`create_model()`で作ったモデルを「コンパイル」しなければならない。コンパイルすることにて、モデルをGPUで実行できるようになる

In [11]:
model.compile(optimizer="sgd", loss="mean_squared_error")

上記の命令を解説すると：

- `optimizer`:「最適化方法」という意味であり、勾配関数のことを表している。`sgd`と`Adam`をよく使われている
- `loss`:損失関数のこと。課題により、適する損失関数がありが、とりあえず引き続きに「平均二乗誤差」を利用

なお、学習は、以下の命令で実装：

`model.fit(x, y_true, batch_size=1, epochs=1500)`

- `x`:入力データ
- `y_true`:正しい答えとして期待しているラベル
- `epochs`:学習する回数
- `batch_size`:「バッチサイズ」は、入力データを何点を一緒にまとめて学習させるのか

In [12]:
# 学習データを用意
y_true = np.array([
    [0],
    [0],
    [0],
    [1]
], dtype = float)


model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.2484  
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2144 
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.3311 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2671 
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2239 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.1941 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2598 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2927 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2200 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2469 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.2305
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2255
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2430
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.2171
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.2195 
Ep

In [13]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
[[0.09739805]
 [0.31559506]
 [0.27350143]
 [0.6166743 ]]


## 改善方法：適する損失関数を選ぶ

「平均二乗誤差」普段に「距離」が表している（正解数値からどれぐらい離れるのか？）。実数の場合はこれで良いと思っていいのが、学習データはバイナリ（0か1）の場合、kerasがもと良い損失関数を提供する。それは`binary_crossentropy`という。

それでは、関数を変えて、確かめてみよう～

In [14]:
# もう一度乱数を固定、モデルを構築する
model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [15]:
# 学習実行
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.7635  
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7731 
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2118 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2345 
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6883 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6020 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9363 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1607 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6892 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5761 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9127 
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9014 
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6295 
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9117 
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8686 
Epo

In [16]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
[[0.01788807]
 [0.18373643]
 [0.18319982]
 [0.73487896]]


In [18]:
print(model.get_weights())

[array([[2.5107543],
       [2.5143363]], dtype=float32), array([-4.0055714], dtype=float32)]


## 練習：ORを学習させる

In [19]:
# 学習データを用意
y_true = np.array([
    [0],
    [1],
    [1],
    [1]
], dtype = float)

model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5787  
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5299 
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5860 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5720 
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5935 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5905 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5000 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5662 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5646 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4796 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4858 
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4421 
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5044 
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5799 
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4748 
Epo

In [20]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
[[0.26742923]
 [0.899479  ]
 [0.8989957 ]
 [0.9954373 ]]


In [21]:
# 加重を確認
print(model.get_weights())

[array([[3.1938202],
       [3.1991537]], dtype=float32), array([-1.0077051], dtype=float32)]


## 練習：XORを学習させる

In [22]:
# 学習データを用意
y_true = np.array([
    [0],
    [1],
    [1],
    [0]
], dtype = float)

model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")
model.fit(x, y_true, batch_size=1, epochs=1500)

Epoch 1/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.6616  
Epoch 2/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8834 
Epoch 3/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5529 
Epoch 4/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6589 
Epoch 5/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6581 
Epoch 6/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6719 
Epoch 7/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9838 
Epoch 8/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7306 
Epoch 9/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5662 
Epoch 10/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.2360 
Epoch 11/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5551 
Epoch 12/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9517 
Epoch 13/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7780 
Epoch 14/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9453 
Epoch 15/1500
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5694 
Epo

In [23]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
[[0.48164088]
 [0.49630097]
 [0.49766302]
 [0.5123287 ]]
